# Calculations for Machine Vision

요구 사항이 다음과 같다고 하자.  
검사 대상의 크기 : 가로 40mm x 세로 40mm  
검사(검출)해야하는 최소 불량 크기 : 40um  
검사 대상으로부터 카메라 렌즈 끝단까지의 거리 : 100 ~ 200mm 사이 * 이 것은 검사 환경의 제약에 의해 요구되는 것으로 WD(Working Distance)라고 한다.

## 1. 필요 분해능
최소 불량 크기를 40um라고 하자. 그러면 그 것을 검출하기 위해서는 일반적으로 최소 2보다 큰 수로 나눈 분해능이 필요하다.  
(여기서 2에 해당하는 값을 MR이라 부르는데, 최소 값이 2가 되는 것은 나이키스트 샘플링 이론에 근거한다.)

In [1]:
resol = 20 # 이것은 카메라(이미지 센서)의 한 픽셀이 보는 크기이므로 PSS(Pixel Sampling Size)라고도 한다.

## 2. 카메라(이미지 센서)의 선택

## 2.1 카메라의 화면 해상도

In [2]:
# 카메라 해상도 = 검사 대상의 크기(40000) / 필요 분해능(20)
min_cam_resol = 40000 / 20 # 2000 pixels
# 머신 비전 카메라 중 5Mega Mono(화면 해상도 : 2448x2048)를 선택하자.
cam_resol_x = 2448
cam_resol_y = 2048

## 2.2 카메라(이미지 센서)의 픽셀 크기

In [3]:
# 일반적으로, 머신 비전에서 많이 사용하는 이미지 센서의 크기는 1/2", 2/3", 1/3", 1/4" 등 이다.
# 여기서는 2/3"를 선택한다. (가로 8.8mm x 세로 6.6mm. 대각선 11mm임)
cam_pixel_size = 8.8 * 1000 / cam_resol_x # [um]
# 또는, cam_pixel_size = 6.6 * 1000 / cam_resol_y # [um]
cam_pixel_size

3.5947712418300655

## 3. Lens의 선택

## 3.1 배율(m)
a : 물체에서 렌즈까지의 거리  
b : 렌즈에서 상(Image)까지의 거리  
m : 배율 (m = b/a)

In [4]:
m = cam_pixel_size / resol
m

0.17973856209150327

## 3.2 초점거리(f)

In [11]:
# 렌즈의 공식 (1/a + 1/ b = 1/f)로 부터 f = a(m/(1+m))이 유도된다.
# 또한, a = k*f라고 하면 (즉 a를 f의 몇 배 거리로 나타낼 때) k = (1+m)/m이 유도된다.

In [5]:
k = (1+m) / m
k # k는 a가 f의 몇 배인지를 나타냄

6.5636363636363635

In [6]:
# 또한, 대략적으로 m = f / WD가 되므로, f = m * WD 로 구할 수 있다.
# WD는 100 ~ 200 사이라는 요구 사항에 맞게 선택한다. 여기서는 170으로 한다.
WD = 170 # [mm]
f = m * WD # [mm]
f

30.555555555555557

In [8]:
# a(물체에서 렌즈까지의 거리)는 WD보다 길다. 따라서 경통의 길이 정도를 더해준다.
a = k * f
a

200.55555555555557

In [9]:
b = a * m
b

36.04756717501816

f는 약 30.6mm, b는 약 36.0mm로 상(Image)이 초점보다 뒤에 생기는 것을 알 수 있다.

## 3.3 렌즈의 NA와 분해능
NA (Numerical Aperture)  
Lens Resolution : $R = k\frac{\lambda}{NA}$

In [10]:
# 렌즈는 NA가 제작 사영으로 정해진다.
# NA가 0.017인 렌즈를 선택했다고 하자.

In [11]:
NA = 0.017

In [12]:
# 렌즈의 분해능은 다음과 같다.
# Lens Resol = k*lambda/NA (k는 렌즈의 특성 상수(보통 0.6), lambda는 빛의 파장인데, 보통 녹색광(550[nm])를 기준으로 한다.)
lens_resol = 0.6 * 0.55 / NA
lens_resol

19.41176470588235

렌즈의 분해능이 약 19.4[um]로 필요 분해능(reol = 20[um])보다 작게 나왔다. (실제로는 이 조건에 맞는 NA를 가진 렌즈를 선택하는 것임)  
만약 필요 분해능보다 크게 나온다면 그 렌즈는 사용할 수 없다.

## 3.4 렌즈의 EA와 F#
EA (Effective Aperture) : $EA = \frac{n\cdot m}{2\cdot NA}$  
F# (F Number) : $F\# = \frac{f}{D} = \frac{EA}{(1+m)}$

In [13]:
EA = 1 * m / (2 * NA)
EA

5.28642829680892

In [14]:
Fnum = EA / (1 + m)
Fnum

4.481016783444679

In [15]:
D = f / Fnum
D

6.81888888888889

## 3.5 렌즈의 초점심도(DOF)
DOF (Depth Of Focus) : $DOF = \frac{2\cdot CC \cdot EA}{m^2}$ 

In [16]:
DOF = 2 * resol * EA / m**2
DOF

6545.454545454546

DOF가 약 6.5[mm]가 나왔다. 이 것은 검사 대상이 WD인 170[mm]보다 +/-3.25[mm] 정도의 위치 편차가 있어도 포커스가 맞는다는 의미이다.